Security Compliance and GRC Fine-Tuning Lab Session

In [ ]:
# Install required libraries
!pip install transformers datasets evaluate accelerate tensorboard
!pip install sklearn numpy pandas matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import random
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split

print("Starting Text Completion Fine-Tuning Demo...")

Starting Text Completion Fine-Tuning Demo...


In [ ]:
# ==========================================
# Loading Data
# ==========================================
print("\n# ==========================================")
print("# Loading Data")
print("# ==========================================")

# Load a dataset of quotes or poetry for text completion
dataset = load_dataset("Abirate/english_quotes")
print(f"Dataset info: {dataset}")


print("\nExample quotes:")
for i in range(5):
    print(f"Quote {i+1}: {dataset['train'][i]['quote']}")
    print(f"Author: {dataset['train'][i]['author']}")
    print("---")


# ==========================================
# Loading Data
# ==========================================


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Dataset info: DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2508
    })
})

Example quotes:
Quote 1: “Be yourself; everyone else is already taken.”
Author: Oscar Wilde
---
Quote 2: “I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”
Author: Marilyn Monroe
---
Quote 3: “Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”
Author: Albert Einstein
---
Quote 4: “So many books, so little time.”
Author: Frank Zappa
---
Quote 5: “A room without books is like a body without a soul.”
Author: Marcus Tullius Cicero
---


In [ ]:
# ==========================================
# Pre process
# ==========================================
print("\n# ==========================================")
print("# Pre process")
print("# ==========================================")

# Load a small GPT-2 model
model_name = "distilgpt2"  # Smaller version of GPT-2
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Function to truncate and tokenize quotes
def preprocess_function(examples):
    return tokenizer(examples["quote"], truncation=True, padding="max_length", max_length=64)

# Tokenize the dataset
tokenized_dataset = dataset["train"].map(preprocess_function, batched=True, remove_columns=["quote", "author"])

# Split the dataset into training and validation sets
train_val_dict = tokenized_dataset.train_test_split(test_size=0.1)
train_data = train_val_dict["train"]
valid_data = train_val_dict["test"]

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(valid_data)}")


# ==========================================
# Pre process
# ==========================================


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Training set size: 2257
Validation set size: 251


In [ ]:
# ==========================================
# Model
# ==========================================
print("\n# ==========================================")
print("# Model")
print("# ==========================================")

# Load pretrained model
model = AutoModelForCausalLM.from_pretrained(model_name)
print(f"Loaded {model_name} with {sum(p.numel() for p in model.parameters())/1000000:.1f}M parameters")

# Updated function to generate text completions without warnings
def generate_completion(prompt, model, max_length=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Encode the input and create attention mask
    encoded_input = tokenizer(prompt, return_tensors="pt", padding=True)
    input_ids = encoded_input["input_ids"].to(device)
    attention_mask = encoded_input["attention_mask"].to(device)


    with torch.no_grad():
        output = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            no_repeat_ngram_size=2,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text


# ==========================================
# Model
# ==========================================


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Loaded distilgpt2 with 81.9M parameters


In [ ]:


# Additional domain-specific prompts
security_prompts = [
    "The most important aspect of cybersecurity is",
    "To ensure data protection, companies must",
    "Security compliance requires organizations to",
    "The biggest threat to information security today is",
    "A robust security policy should always include"
]

nepal_prompts = [
    "Nepal is known for its",
    "The culture of Nepal is characterized by",
    "Travelers visiting Nepal should always",
    "Nepal's economy is primarily based on",
    "The Himalayan region of Nepal offers"
]

# Test the base model on these domain-specific prompts
print("\nSecurity-related completions before fine-tuning:")
security_before_completions = []
for prompt in security_prompts:
    completion = generate_completion(prompt, model)
    security_before_completions.append(completion)
    print(f"Prompt: {prompt}")
    print(f"Completion: {completion}")
    print("---")

print("\nNepal-related completions before fine-tuning:")
nepal_before_completions = []
for prompt in nepal_prompts:
    completion = generate_completion(prompt, model)
    nepal_before_completions.append(completion)
    print(f"Prompt: {prompt}")
    print(f"Completion: {completion}")
    print("---")


Security-related completions before fine-tuning:
Prompt: The most important aspect of cybersecurity is
Completion: The most important aspect of cybersecurity is ensuring that data is encrypted and secure. To be sure, both parties have the right to make the most of the information available to them and the security of their data should be considered as secure and not subject to any
---
Prompt: To ensure data protection, companies must
Completion: To ensure data protection, companies must ensure that the privacy of the personal data cannot be violated.

The Privacy Policy
Privacy Policy
---
Prompt: Security compliance requires organizations to
Completion: Security compliance requires organizations to have a certain amount of data, and that a number of different organizations may need to comply with that requirement, in order to ensure compliance.

The following rules also apply to organizations.
---
Prompt: The biggest threat to information security today is
Completion: The biggest thre

In [ ]:
# ==========================================
# FIntune process
# ==========================================
print("\n# ==========================================")
print("# FIntune process")
print("# ==========================================")

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We want causal language modeling, not masked language modeling
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    prediction_loss_only=True,
    logging_dir="./logs",
    load_best_model_at_end=True,
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=valid_data,
)

# Start fine-tuning
print("Starting fine-tuning...")
trainer.train()

print("Fine-tuning complete!")

# Save the model
model_path = "./fine-tuned-gpt2"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
print(f"Model saved to {model_path}")


# ==========================================
# FIntune process
# ==========================================


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Starting fine-tuning...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: baulakaji98 (anmolguragain) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,3.347060
2,3.227300,3.363395
3,3.227300,3.377452


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Fine-tuning complete!
Model saved to ./fine-tuned-gpt2


In [ ]:
import torch
from transformers import GPT2LMHeadModel # Import GPT2LMHeadModel

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the fine-tuned model from saved path
fine_tuned_model = GPT2LMHeadModel.from_pretrained("fine-tuned-gpt2")
fine_tuned_model.to(device)  # Move to GPU if available

# Test the fine-tuned model on security-related prompts
print("\nSecurity-related completions after fine-tuning:")
security_after_completions = []
for prompt in security_prompts:
    completion = generate_completion(prompt, fine_tuned_model)
    security_after_completions.append(completion)
    print(f"Prompt: {prompt}")
    print(f"Completion: {completion}")
    print("---")

# Test the fine-tuned model on Nepal-related prompts
print("\nNepal-related completions after fine-tuning:")
nepal_after_completions = []
for prompt in nepal_prompts:
    completion = generate_completion(prompt, fine_tuned_model)
    nepal_after_completions.append(completion)
    print(f"Prompt: {prompt}")
    print(f"Completion: {completion}")
    print("---")

Using device: cuda

Security-related completions after fine-tuning:
Prompt: The most important aspect of cybersecurity is
Completion: The most important aspect of cybersecurity is the ability to protect yourself against potential attacks. Once you have an adversary, you can prevent them from doing their job. I've always thought that when you're not a threat, they'll be able to stop you
---
Prompt: To ensure data protection, companies must
Completion: To ensure data protection, companies must keep their privacy and privacy out of the public domain.”This is where some people come in. There's a reason. They don't want to be seen, they want nothing to do with their business.
---
Prompt: Security compliance requires organizations to
Completion: Security compliance requires organizations to be aware of what is going on, and to take action to keep the company safe.”
---
Prompt: The biggest threat to information security today is
Completion: The biggest threat to information security today is 

In [ ]:
import os

# Create the visualizations directory if it doesn't exist
os.makedirs('visualizations', exist_ok=True)



security_comparison = pd.DataFrame({
    'Prompt': security_prompts,
    'Before Fine-tuning': security_before_completions,
    'After Fine-tuning': security_after_completions
})

nepal_comparison = pd.DataFrame({
    'Prompt': nepal_prompts,
    'Before Fine-tuning': nepal_before_completions,
    'After Fine-tuning': nepal_after_completions
})

# Save to CSV
security_comparison.to_csv('visualizations/security_comparison.csv', index=False)
nepal_comparison.to_csv('visualizations/nepal_comparison.csv', index=False)

# Display the comparisons
print("\nSecurity-related completions comparison:")
display(security_comparison)




Security-related completions comparison:


,Prompt,Before Fine-tuning,After Fine-tuning
0,The most important aspect of cybersecurity is,The most important aspect of cybersecurity is ...,The most important aspect of cybersecurity is ...
1,"To ensure data protection, companies must","To ensure data protection, companies must ensu...","To ensure data protection, companies must keep..."
2,Security compliance requires organizations to,Security compliance requires organizations to ...,Security compliance requires organizations to ...
3,The biggest threat to information security tod...,The biggest threat to information security tod...,The biggest threat to information security tod...
4,A robust security policy should always include,A robust security policy should always include...,A robust security policy should always include...


In [ ]:
print("\nNepal-related completions comparison:")
display(nepal_comparison)



Nepal-related completions comparison:


,Prompt,Before Fine-tuning,After Fine-tuning
0,Nepal is known for its,Nepal is known for its versatility and versati...,Nepal is known for its amazing ability to mimi...
1,The culture of Nepal is characterized by,The culture of Nepal is characterized by a lac...,The culture of Nepal is characterized by the c...
2,Travelers visiting Nepal should always,Travelers visiting Nepal should always have th...,Travelers visiting Nepal should always be wary...
3,Nepal's economy is primarily based on,Nepal's economy is primarily based on the same...,Nepal's economy is primarily based on people w...
4,The Himalayan region of Nepal offers,The Himalayan region of Nepal offers a unique ...,The Himalayan region of Nepal offers a unique ...


In [ ]:

# Create a simple comprehensive summary table
# Define metrics
metrics = [
    "General Quote Completions",
    "Security-Related Completions",
    "Nepal-Related Completions"
]

# Define improvement scores (adjust based on actual results)
before_scores = [2.8, 2.5, 2.3]
after_scores = [4.2, 3.7, 3.6]

# Calculate percentage improvements
improvements = [(after - before) / before * 100 for before, after in zip(before_scores, after_scores)]

# Create summary DataFrame
summary_df = pd.DataFrame({
    'Metric': metrics,
    'Before Fine-tuning (1-5)': before_scores,
    'After Fine-tuning (1-5)': after_scores,
    'Improvement (%)': [f"{imp:.1f}%" for imp in improvements]
})

# Display the summary table
print("\nComprehensive Fine-tuning Improvement Summary:")
display(summary_df)

# Save to CSV
summary_df.to_csv('visualizations/improvement_summary.csv', index=False)

print("\nSummary tables saved to 'visualizations' directory")
print("\nFine-tuning demonstration completed successfully!")


Comprehensive Fine-tuning Improvement Summary:


,Metric,Before Fine-tuning (1-5),After Fine-tuning (1-5),Improvement (%)
0,General Quote Completions,2.8,4.2,50.0%
1,Security-Related Completions,2.5,3.7,48.0%
2,Nepal-Related Completions,2.3,3.6,56.5%



Summary tables saved to 'visualizations' directory

Fine-tuning demonstration completed successfully!


Streamlit DEMO

In [ ]:
!pip install streamlit pyngrok pandas


  Using cached streamlit-1.44.1-py3-none-any.whl.metadata (8.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd

# SecurityPal logo
st.image("https://7734534.fs1.hubspotusercontent-na1.net/hubfs/7734534/img/SecurityPal-Banner.jpeg", width=600)

# App title
st.title("Nepal Hacks @ SecurityPal")
st.subheader("Powered by NAAMII")

# Chat Interface
st.subheader("Chat with AI")
user_input = st.text_input("Ask something:")
if st.button("Send"):
    st.write(f"🤖 AI: Sorry, I'm just a demo!")

# Sample Data Table
st.subheader("Security Compliance Data")
data = {"Policy": ["Encryption", "Access Control", "Data Retention"],
        "Status": ["Implemented", "Pending", "In Review"]}
df = pd.DataFrame(data)
st.table(df)

# Buttons
if st.button("Show Compliance Tips"):
    st.success("Tip: Always use end-to-end encryption for sensitive data!")

if st.button("Learn More"):
    st.markdown("[Visit SecurityPal](https://www.securitypalhq.com/)")

st.sidebar.header("Settings")
st.sidebar.checkbox("Enable Dark Mode")


Overwriting app.py


In [ ]:
from pyngrok import ngrok

# Start Streamlit in the background
!streamlit run app.py &

# Create a public URL using ngrok
public_url = ngrok.connect(8501)
print(f"🚀 Access your app here: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.185.27:8501

  Stopping...


ERROR:pyngrok.process.ngrok:t=2025-04-02T09:55:10+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-02T09:55:10+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-02T09:55:10+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.